In [ ]:
import cv2
import numpy as np
from sklearn.svm import SVC
from sklearn.externals import joblib
from cv2 import HOGDescriptor
import LineDetector
from numpy.linalg import norm

class PathFinder: 
    
    
    def __init__(self,model_signs,model_semafors,printFlag,video_source=0,
                 resize=(640,480)):
        self.clfSigns = joblib.load(model_signs) 
        self.clfSem = joblib.load(model_semafors)         
        self.cap=cv2.VideoCapture(video_source)
        self.size = resize
        self.printFlag = printFlag

            
    def __hog2(self,image):
        winSize = (16,48)
        blockSize = (16,16)
        blockStride = (8,8)
        cellSize = (8,8)
        nbins = 9
        derivAperture = 1
        winSigma = 4.
        histogramNormType = 0
        L2HysThreshold = 2.0000000000000001e-01
        gammaCorrection = 100
        nlevels = 32
        hog = HOGDescriptor(winSize,blockSize,blockStride,cellSize,nbins,derivAperture,winSigma,
                        histogramNormType,L2HysThreshold,gammaCorrection,nlevels)
        winStride = (8,8)
        padding = (16,16)
        hist = hog.compute(image,winStride,padding)
        return np.reshape(hist, 4500)
    
    
    def __hog(self,img,step):
        gx = cv2.Sobel(img, cv2.CV_32F, 1, 0)
        gy = cv2.Sobel(img, cv2.CV_32F, 0, 1)
        mag, ang = cv2.cartToPolar(gx, gy)
        bin_n = 16 # Number of bins
        bin = np.int32(bin_n*ang/(2*np.pi))
        bin_cells = []
        mag_cells = []
        cellx = celly = step   
        for i in range(0,int(img.shape[0]/celly)):
            for j in range(0,int(img.shape[1]/cellx)):
                bin_cells.append(bin[i*celly : i*celly+celly, j*cellx : j*cellx+cellx])
                mag_cells.append(mag[i*celly : i*celly+celly, j*cellx : j*cellx+cellx])   
        hists = [np.bincount(b.ravel(), m.ravel(), bin_n) for b, m in zip(bin_cells, mag_cells)]
        hist = np.hstack(hists)
        eps = 1e-7
        hist /= hist.sum() + eps
        hist = np.sqrt(hist)
        hist /= norm(hist) + eps
        return hist
    
    
    def StartRide(self):
        frame=self.cap.read()[1]
        self.line_detector = LineDetector.RoadControl(frame, ROI_height = 210, viz = True)
        self.line_detector.vec1 = [-3, -1, 70]
        self.line_detector.vec2 = [3, -1, 70]
        
        while(self.cap.isOpened() and len(frame)>0):
            frame = cv2.resize(frame,self.size, interpolation = cv2.INTER_CUBIC)
            predictSigns=self.__detectSigns(frame,self.printFlag)
            predictSemafors = self.__detectSemafors(frame,self.printFlag)
            
            self.line_detector.img = frame
#            print(self.line_detector.poke())
            #HERE ADD YOUR MODULES
            cv2.imshow('frame2',frame)
            if cv2.waitKey(2) & 0xFF == ord("q"):
                break
            frame=self.cap.read()[1]
            
        self.cap.release()
        cv2.destroyAllWindows()
    
    def __detectSemafors(self,frame,printFlag):
        frame2 = cv2.GaussianBlur(frame,(3,3),0)
        r,g,b = cv2.split(frame2)
        circles = cv2.HoughCircles(b,cv2.HOUGH_GRADIENT,0.5,50,param1=70,param2=10,minRadius=2,maxRadius=20)
        if np.all(circles) != None:
            circles = np.uint16(np.around(circles))
            for i in circles[0,:]:
                if frame2[i[1]][i[0]][2] > 160 and frame2[i[1]][i[0]][1]<150 and frame2[i[1]][i[0]][0]<150:                    
                    x1=i[0]-i[2]*2
                    y1=i[1]-i[2]*2
                    x2=i[0]+i[2]*2
                    y2=i[1]+i[2]*7
                    if y2>self.size[1]-1:
                        y2=self.size[1]-1
                    if x2>self.size[0]-1:
                        x2=self.size[0]-1
                    if(x1 > 0 and x2 >0 and y1 > 0 and y2 > 0):                     
                        cut_frame = frame2[y1:y2,x1:x2]
                        cut_frame = cv2.resize(cut_frame,(16, 48), interpolation = cv2.INTER_CUBIC)
                        pred = self.clfSem.predict_proba([self.__hog2(cut_frame)])[0][1]
                        if(pred>0.7):    
                            if(printFlag):
                                cv2.circle(frame,(i[0],i[1]),i[2],(0,255,255),2)
                                frame = cv2.rectangle(frame,(x1,y1),(x2,y2),(0,0,255),2)
                            return True
                        return False
    
    def __detectSigns(self,frame,printFlag):
        frame2 = cv2.GaussianBlur(frame,(3,3),0)
        frame2 = cv2.medianBlur(frame,5)
        r,g,b = cv2.split(frame2)    
        circles = cv2.HoughCircles(r,cv2.HOUGH_GRADIENT,0.5,70,param1=250,param2=20,minRadius=10,maxRadius=40) #let's change params
        pred=[]
        if np.all(circles) != None:
            circles = np.round(circles[0, :]).astype("int")
            for (x, y, r) in circles:
                if((x-r) > 0 and (x+r) >0 and (y-r)>0 and (y+r)>0):
                    cut_frame = frame2[y-r:y+r,x-r:x+r]
                    cv2.imshow('frame2',cut_frame)
                    cut_frame = cv2.resize(cut_frame,(20, 20), interpolation = cv2.INTER_CUBIC)
                    if(printFlag):
                        frame = cv2.rectangle(frame,(x-r,y-r),(x+r,y+r),(0,0,255),2)
                    hog=self.__hog(cut_frame,8)
                    pred.append(self.clfSigns.predict([hog])) #change to clf.predict_proba              
        return pred
    
    
    

In [ ]:
One = PathFinder('model.pkl','semafor/model_semafors.pkl',False,"output.avi")
One.StartRide()